In [1]:
!pip install wandb

In [2]:
import wandb
key = input('Enter your API:')
wandb.login(key=key)

Enter your API: 580e769ee2f34eafdded556ce52aaf31c265ad3b


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ma23m011 (ma23m011-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import wandb
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd

# Dataset 
class TransliterationDataset(Dataset):
    def __init__(self, data, src_vocab, tgt_vocab):
        self.data = data
        self.src_vocab = src_vocab
        self.tgt_vocab = tgt_vocab

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        src, tgt = self.data[idx]
        # for c in src:
        #     print(c)
        src_ids = [self.src_vocab['<sos>']] + [self.src_vocab.get(c, self.src_vocab['<unk>']) for c in src] + [self.src_vocab['<eos>']]
        tgt_ids = [self.tgt_vocab['<sos>']] + [self.tgt_vocab.get(c, self.tgt_vocab['<unk>']) for c in tgt] + [self.tgt_vocab['<eos>']]
        return torch.tensor(src_ids), torch.tensor(tgt_ids)

def collate_fn(batch):
    src_seqs, tgt_seqs = zip(*batch)
    src_padded = nn.utils.rnn.pad_sequence(src_seqs, batch_first=True, padding_value=0)
    tgt_padded = nn.utils.rnn.pad_sequence(tgt_seqs, batch_first=True, padding_value=0)
    return src_padded, tgt_padded

# Vocab 
def build_vocab(data):
    vocab = {'<pad>': 0, '<sos>': 1, '<eos>': 2, '<unk>': 3}
    idx = 4
    for word in data:
        if isinstance(word, str):
            for char in word:
                if char not in vocab:
                    vocab[char] = idx
                    idx += 1
    return vocab

# Model
class Seq2Seq(nn.Module):
    def __init__(self, config, src_vocab_size, tgt_vocab_size):
        super().__init__()
        self.config = config
        self.embedding_src = nn.Embedding(src_vocab_size, config['embedding_dim'], padding_idx=0)
        self.embedding_tgt = nn.Embedding(tgt_vocab_size, config['embedding_dim'], padding_idx=0)

        rnn_cell = {'RNN': nn.RNN, 'GRU': nn.GRU, 'LSTM': nn.LSTM}[config['cell_type']]
        self.encoder = rnn_cell(config['embedding_dim'], config['hidden_size'], config['encoder_layers'], dropout=config['dropout'], batch_first=True)
        self.decoder = rnn_cell(config['embedding_dim'], config['hidden_size'], config['decoder_layers'], dropout=config['dropout'], batch_first=True)
        
        self.fc_out = nn.Linear(config['hidden_size'], tgt_vocab_size)

    def forward(self, src, tgt):
        embedded_src = self.embedding_src(src)
        embedded_tgt = self.embedding_tgt(tgt)

        _, hidden = self.encoder(embedded_src)

        def expand_hidden(h_enc, required_layers):
            # h_enc: (num_layers_enc, batch, hidden_size)
            num_enc_layers = h_enc.size(0)
            if num_enc_layers < required_layers:
                # Create additional zero layers
                extra = torch.zeros(
                    required_layers - num_enc_layers,
                    h_enc.size(1),
                    h_enc.size(2),
                    device=h_enc.device,
                    dtype=h_enc.dtype
                )
                h_enc = torch.cat([h_enc, extra], dim=0)
            else:
                h_enc = h_enc[-required_layers:]  # if more, take only needed
            return h_enc
        
        if isinstance(hidden, tuple):  # LSTM
            h, c = hidden
            h = expand_hidden(h, self.config['decoder_layers'])
            c = expand_hidden(c, self.config['decoder_layers'])
            decoder_output, _ = self.decoder(embedded_tgt, (h, c))
        else:  # GRU / RNN
            hidden = expand_hidden(hidden, self.config['decoder_layers'])
            decoder_output, _ = self.decoder(embedded_tgt, hidden)

        output = self.fc_out(decoder_output)
        return output

#  Accuracy 
def calculate_accuracy(output, target, pad_idx):
    preds = output.argmax(2)
    mask = (target != pad_idx)
    correct = (preds == target) & mask
    return correct.sum().item() / mask.sum().item()

def compute_word_accuracy(output, target, tgt_index_to_token, pad_idx):
    preds = output.argmax(dim=2)  # [batch, seq_len]
    correct = 0
    total = 0

    for pred_seq, tgt_seq in zip(preds, target):
        # Convert to tokens
        pred_tokens = [tgt_index_to_token[idx.item()] for idx in pred_seq if idx.item() != pad_idx]
        tgt_tokens = [tgt_index_to_token[idx.item()] for idx in tgt_seq if idx.item() != pad_idx]

        # Remove after <eos> if exists
        if '<eos>' in pred_tokens:
            pred_tokens = pred_tokens[:pred_tokens.index('<eos>')]
        if '<eos>' in tgt_tokens:
            tgt_tokens = tgt_tokens[:tgt_tokens.index('<eos>')]

        if pred_tokens == tgt_tokens:
            correct += 1
        total += 1

    return correct / total if total > 0 else 0.0

# Training
def train(model, dataloader, optimizer, criterion, tgt_pad_idx, tgt_index_to_token):
    model.train()
    total_loss, total_acc, total_word_acc = 0, 0, 0
    for src, tgt in dataloader:
        src, tgt = src.to(device), tgt.to(device)
        optimizer.zero_grad()
        output = model(src, tgt[:, :-1])
        loss = criterion(output.reshape(-1, output.shape[-1]), tgt[:, 1:].reshape(-1))
        acc = calculate_accuracy(output, tgt[:, 1:], tgt_pad_idx)
        word_acc = compute_word_accuracy(output, tgt[:, 1:], tgt_index_to_token, tgt_pad_idx)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        total_acc += acc
        total_word_acc += word_acc
    return total_loss / len(dataloader), total_acc / len(dataloader), total_word_acc / len(dataloader)

def evaluate(model, dataloader, criterion, tgt_pad_idx, tgt_index_to_token):
    model.eval()
    total_loss, total_acc, total_word_acc = 0, 0, 0
    with torch.no_grad():
        for src, tgt in dataloader:
            src, tgt = src.to(device), tgt.to(device)
            output = model(src, tgt[:, :-1])
            loss = criterion(output.reshape(-1, output.shape[-1]), tgt[:, 1:].reshape(-1))
            acc = calculate_accuracy(output, tgt[:, 1:], tgt_pad_idx)
            word_acc = compute_word_accuracy(output, tgt[:, 1:], tgt_index_to_token, tgt_pad_idx)
            total_loss += loss.item()
            total_acc += acc
            total_word_acc += word_acc
    return total_loss / len(dataloader), total_acc / len(dataloader), total_word_acc / len(dataloader)


#  Sweep  
def sweep_train():
    wandb.init()
    config = wandb.config
    train_df = pd.read_csv("/kaggle/input/dfefeff/dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.train.tsv", sep="\t", header=None, names=["tgt", "src", "freq"])
    dev_df = pd.read_csv("/kaggle/input/dfefeff/dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.dev.tsv", sep="\t", header=None, names=["tgt", "src", "freq"])
    train_df['src'] = train_df['src'].astype(str)
    train_df['tgt'] = train_df['tgt'].astype(str)

    src_vocab = build_vocab(train_df['src'])
    tgt_vocab = build_vocab(train_df['tgt'])
    print(src_vocab)
    print(tgt_vocab)
    tgt_index_to_token = {v: k for k, v in tgt_vocab.items()}

    train_data = list(zip(train_df['src'], train_df['tgt']))
    dev_data = list(zip(dev_df['src'], dev_df['tgt']))

    train_dataset = TransliterationDataset(train_data, src_vocab, tgt_vocab)
    dev_dataset = TransliterationDataset(dev_data, src_vocab, tgt_vocab)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
    dev_loader = DataLoader(dev_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

    model = Seq2Seq(config, len(src_vocab), len(tgt_vocab)).to(device)
    optimizer = optim.Adam(model.parameters())
    criterion = nn.CrossEntropyLoss(ignore_index=tgt_vocab['<pad>'])

    for epoch in range(config['epochs']):
        train_loss, train_acc, train_word_acc = train(model, train_loader, optimizer, criterion, tgt_vocab['<pad>'], tgt_index_to_token)
        val_loss, val_acc, val_word_acc = evaluate(model, dev_loader, criterion, tgt_vocab['<pad>'], tgt_index_to_token)
        # #print({
        #     "epoch": epoch + 1,
        #     "train_loss": train_loss,
        #     "val_loss": val_loss,
        #     "train_accuracy": train_acc * 100,
        #     "val_accuracy": val_acc * 100,
        #     "train_word_accuracy": train_word_acc * 100,
        #     "val_word_accuracy": val_word_acc * 100
        # })
        print(f"Epoch {epoch + 1}")
        print(f"{'train_loss:':20} {train_loss:.4f}")
        print(f"{'val_loss:':20} {val_loss:.4f}")
        print(f"{'train_accuracy:':20} {train_acc * 100:.2f}%")
        print(f"{'val_accuracy:':20} {val_acc * 100:.2f}%")
        print(f"{'train_word_accuracy:':20} {train_word_acc * 100:.2f}%")
        print(f"{'val_word_accuracy:':20} {val_word_acc * 100:.2f}%")
        wandb.log({
            "epoch": epoch + 1,
            "train_loss": train_loss,
            "val_loss": val_loss,
            "train_accuracy": train_acc * 100,
            "val_accuracy": val_acc * 100,
            "train_word_accuracy": train_word_acc * 100,
            "val_word_accuracy": val_word_acc * 100
        })


#  Run 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

sweep_config = {
    'method': 'random',
    'name': 'DakshinaSweep_without_attention-8',
    'metric': {'name': 'val_accuracy', 'goal': 'maximize'},
    'parameters': {
        'embedding_dim': {'values': [16, 32, 64, 256]},
        'hidden_size': {'values': [16, 32, 64, 256]},
        'encoder_layers': {'values': [1,2,3]},
        'decoder_layers': {'values': [1,2,3]},
        'cell_type': {'values': ['RNN','GRU', 'LSTM']},
        'dropout': {'values': [0.2, 0.3]},
        'epochs': {'values': [5,10,13,15,17,20]}
    }
}

sweep_id = wandb.sweep(sweep_config, project="DL_A3")
wandb.agent(sweep_id, function=sweep_train, count = 50)

Create sweep with ID: xcy3vfdz
Sweep URL: https://wandb.ai/ma23m011-iit-madras/DL_A3/sweeps/xcy3vfdz


wandb: Agent Starting Run: fx3ux0zg with config:
wandb: 	cell_type: GRU
wandb: 	decoder_layers: 2
wandb: 	dropout: 0.3
wandb: 	embedding_dim: 32
wandb: 	encoder_layers: 3
wandb: 	epochs: 15
wandb: 	hidden_size: 16
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


{'<pad>': 0, '<sos>': 1, '<eos>': 2, '<unk>': 3, 'a': 4, 'n': 5, 'g': 6, 'k': 7, 'i': 8, 't': 9, 'o': 10, 'e': 11, 'r': 12, 's': 13, 'h': 14, 'y': 15, 'w': 16, 'u': 17, 'l': 18, 'd': 19, 'j': 20, 'b': 21, 'm': 22, 'c': 23, 'q': 24, 'z': 25, 'p': 26, 'x': 27, 'v': 28, 'f': 29}
{'<pad>': 0, '<sos>': 1, '<eos>': 2, '<unk>': 3, 'অ': 4, 'ং': 5, 'ক': 6, 'ি': 7, 'ত': 8, 'ে': 9, 'র': 10, 'শ': 11, 'ই': 12, 'ও': 13, 'গ': 14, 'ু': 15, 'ল': 16, 'ো': 17, '্': 18, 'হ': 19, 'ণ': 20, 'া': 21, 'ী': 22, 'দ': 23, 'ন': 24, 'ট': 25, 'ব': 26, 'ষ': 27, 'ম': 28, 'স': 29, 'খ': 30, 'য': 31, 'ড': 32, 'ৎ': 33, 'ধ': 34, 'ঠ': 35, 'জ': 36, 'প': 37, 'ূ': 38, 'চ': 39, 'ছ': 40, 'ভ': 41, 'ঘ': 42, 'ঙ': 43, 'ৈ': 44, 'ঞ': 45, '়': 46, 'ঃ': 47, 'এ': 48, 'থ': 49, 'ৃ': 50, 'ৌ': 51, 'ফ': 52, 'ঝ': 53, 'আ': 54, 'উ': 55, 'ঁ': 56, 'ঈ': 57, 'ঊ': 58, 'ঋ': 59, 'ঐ': 60, 'ঔ': 61, 'ঢ': 62, '২': 63}
Epoch 1
train_loss:          2.6733
val_loss:            2.3371
train_accuracy:      27.48%
val_accuracy:        34.61%
train_word_accuracy:

epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
train_accuracy,▁▄▅▅▆▆▇▇▇▇▇████
train_loss,█▅▄▄▃▃▂▂▂▂▁▁▁▁▁
train_word_accuracy,▁▁▁▂▃▄▄▅▅▅▆▆▇▇█
val_accuracy,▁▃▃▅▆▆▇▇▇▇▇▇███
val_loss,█▆▅▄▃▃▃▂▂▂▂▁▁▁▁
val_word_accuracy,▁▂▂▂▃▅▅▅▆▆▇▆███
epoch,15
train_accuracy,53.61484
train_loss,1.5279
train_word_accuracy,1.44036


wandb: Agent Starting Run: yuhsd9vk with config:
wandb: 	cell_type: GRU
wandb: 	decoder_layers: 1
wandb: 	dropout: 0.3
wandb: 	embedding_dim: 64
wandb: 	encoder_layers: 2
wandb: 	epochs: 13
wandb: 	hidden_size: 256


...

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
train_accuracy,▁▆▇▇████████████████
train_loss,█▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_word_accuracy,▁▄▆▆▇▇▇▇████████████
val_accuracy,▁▆▆▇██████▇▇█▇▇█▇▇█▇
val_loss,▅▁▂▁▂▃▃▅▅▆▆▆▆▇▇▇▇█▇▇
val_word_accuracy,▁▅▆▇▇▇█▇▇▇▆▆▇▇▇▇▆▆▇▆
epoch,20
train_accuracy,94.12101
train_loss,0.17373
train_word_accuracy,61.8441


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 1ain8ars with config:
wandb: 	cell_type: RNN
wandb: 	decoder_layers: 1
wandb: 	dropout: 0.3
wandb: 	embedding_dim: 64
wandb: 	encoder_layers: 3
wandb: 	epochs: 5
wandb: 	hidden_size: 64


{'<pad>': 0, '<sos>': 1, '<eos>': 2, '<unk>': 3, 'a': 4, 'n': 5, 'g': 6, 'k': 7, 'i': 8, 't': 9, 'o': 10, 'e': 11, 'r': 12, 's': 13, 'h': 14, 'y': 15, 'w': 16, 'u': 17, 'l': 18, 'd': 19, 'j': 20, 'b': 21, 'm': 22, 'c': 23, 'q': 24, 'z': 25, 'p': 26, 'x': 27, 'v': 28, 'f': 29}
{'<pad>': 0, '<sos>': 1, '<eos>': 2, '<unk>': 3, 'অ': 4, 'ং': 5, 'ক': 6, 'ি': 7, 'ত': 8, 'ে': 9, 'র': 10, 'শ': 11, 'ই': 12, 'ও': 13, 'গ': 14, 'ু': 15, 'ল': 16, 'ো': 17, '্': 18, 'হ': 19, 'ণ': 20, 'া': 21, 'ী': 22, 'দ': 23, 'ন': 24, 'ট': 25, 'ব': 26, 'ষ': 27, 'ম': 28, 'স': 29, 'খ': 30, 'য': 31, 'ড': 32, 'ৎ': 33, 'ধ': 34, 'ঠ': 35, 'জ': 36, 'প': 37, 'ূ': 38, 'চ': 39, 'ছ': 40, 'ভ': 41, 'ঘ': 42, 'ঙ': 43, 'ৈ': 44, 'ঞ': 45, '়': 46, 'ঃ': 47, 'এ': 48, 'থ': 49, 'ৃ': 50, 'ৌ': 51, 'ফ': 52, 'ঝ': 53, 'আ': 54, 'উ': 55, 'ঁ': 56, 'ঈ': 57, 'ঊ': 58, 'ঋ': 59, 'ঐ': 60, 'ঔ': 61, 'ঢ': 62, '২': 63}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn(


Epoch 1
train_loss:          2.4589
val_loss:            2.3958
train_accuracy:      31.33%
val_accuracy:        33.09%
train_word_accuracy: 0.00%
val_word_accuracy:   0.00%
Epoch 2
train_loss:          2.2238
val_loss:            2.3283
train_accuracy:      36.63%
val_accuracy:        34.28%
train_word_accuracy: 0.00%
val_word_accuracy:   0.00%
